In [2]:
%pip install scipy
%pip install networkx
%pip install matplotlib
%pip install pandas
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import networkx as nx
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
import random
import seaborn as sb


In [4]:
###################QUESTION 1a####################################
print("Program may take several minutes to complete. Please wait.")
edges = pd.read_csv('lastfm_asia_edges.csv')

g1 = nx.from_pandas_edgelist(edges, 'node_1', 'node_2')
print("Computing Eigenvector Centrality...")
eigenvector_centrality = nx.eigenvector_centrality(g1)
print("Computing Degree Centrality...")
degree_centrality = nx.degree_centrality(g1)
try:
    print("Computing Katz Centrality...")
    katz_centrality = nx.katz_centrality(g1, alpha=0.01)
except nx.PowerIterationFailedConvergence:
    print("Katz centrality could not converge")
print("Computing PageRank Centrality...")
pagerank_centrality = nx.pagerank(g1)
print("Computing Betweenness Centrality...")
betweeness_centrality = nx.betweenness_centrality(g1)
print("Computing Closeness Centrality...")
closeness_centrality = nx.closeness_centrality(g1)
print("Eigenvector Centrality:", eigenvector_centrality)
print("Degree Centrality: ", degree_centrality)
if 'katz_centrality' in locals():
    print("Katz Centrality:", katz_centrality)
print("PageRank Centrality:", pagerank_centrality)
print("Betweenness Centrality:", betweeness_centrality)
print("Closeness Centrality:", closeness_centrality)

########################QUESTION 1b###############################
def calculateCorrelationCoefficents():
    dc_list = list(degree_centrality.values())
    ev_list = list(eigenvector_centrality.values())
    if 'katz_centrality' in locals():
        k_list = list(katz_centrality.values())
    else:
        k_list = None
    pr_list = list(pagerank_centrality.values())
    b_list = list(betweeness_centrality.values())
    c_list = list(closeness_centrality.values())

    r_values = []
    correlations = [
        dc_list,
        ev_list,
        k_list,
        pr_list,
        b_list,
        c_list
    ]
    for _, c1 in enumerate(correlations):
        row = []
        for _, c2 in enumerate(correlations):
            if c1 is not None and c2 is not None:
                r, _ = pearsonr(c1, c2)
                row.append(r)
            else:
                row.append(None)
        r_values.append(row)

    print("Pearson's r values: ")
    for row in r_values:
        print(row)

calculateCorrelationCoefficents()

##########################QUESTION 1c###########################
centrality_data = pd.DataFrame({
    'Degree Centrality': list(degree_centrality.values()),
    'Eigenvector Centrality': list(eigenvector_centrality.values()),
    'Katz Centrality': list(katz_centrality.values()) if 'katz_centrality' in locals() else None,
    'PageRank Centrality': list(pagerank_centrality.values()),
    'Betweenness Centrality': list(betweeness_centrality.values()),
    'Closeness Centrality': list(closeness_centrality.values())
})

plt.figure(figsize=(10, 6))
sb.set(style="whitegrid")
sb.ecdfplot(data=centrality_data, marker="x", palette="bright")

plt.legend(title='Centrality Measures', loc='lower right', labels=list(centrality_data.columns))
plt.xlabel('Centrality Value')
plt.ylabel('Cumulative Probability')
plt.ylim(-0.25, 1.25)
plt.xlim(-0.1, 0.3)
plt.title('Cumulative distribution functions')

plt.show()


Program may take several minutes to complete. Please wait.
Computing Eigenvector Centrality...
Computing Degree Centrality...
Computing Katz Centrality...
Computing PageRank Centrality...
Computing Betweenness Centrality...


In [ ]:
#######################QUESTION 2#############################
nodes = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6']
edges = [('v1', 'v3'), ('v3', 'v1'), ('v1', 'v4'), ('v2', 'v1'), ('v3', 'v2'), ('v4', 'v3'), ('v2', 'v5'), ('v5', 'v4'), ('v5', 'v6'), ('v6', 'v2')]
g2 = nx.DiGraph(edges)
adjacency_list = {node: [] for node in nodes}
print(adjacency_list)
for edge in edges:
    source, target = edge
    adjacency_list[source].append(target)
print(adjacency_list)

alpha = 0.85
epsilon = 1e-6
beta = [0.3, 0.2, 0.1, 0.1, 0.1, 0.4]
# the concepts of calculate_pagerank comes from the documentation from networkx pagerank.
def calculate_pagerank(pagerank, iteration=0):
    new_pagerank = [0.0] * 6
    for i in range(6):
        _sum = 0
        for j in range(6):
            if nodes[j] in adjacency_list[nodes[i]]:
                out_degree_j = len(adjacency_list[nodes[j]])
                _sum += pagerank[j] / out_degree_j
        
        new_pagerank[i] = (1 - alpha) + alpha * _sum * beta[i]

    if iteration > 0 and all(abs(new_pagerank[i] - pagerank[i]) < epsilon for i in range(6)):
        #The function must have converged to a stationary point so we just return the new_pagerank
        return new_pagerank
    else:
        #recursively call the function again since it has not fully converged yet.
        return calculate_pagerank(new_pagerank, iteration + 1)
normalize = sum(beta)
beta = [value / normalize for value in beta]
final_pagerank = calculate_pagerank(beta)


print("PageRank Results:")
for i, score in enumerate(final_pagerank):
    print(f"{nodes[i]}: {score}")
beta_dict = {nodes[i]: beta[i] for i in range(len(nodes))}

nx_pagerank = nx.pagerank(g2, alpha=alpha,dangling=beta_dict)
print("\nNetworkx PageRank Results:")
for node, score in nx_pagerank.items():
    print(f"{node}: {score}")



{'v1': [], 'v2': [], 'v3': [], 'v4': [], 'v5': [], 'v6': []}
{'v1': ['v3', 'v4'], 'v2': ['v1', 'v5'], 'v3': ['v1', 'v2'], 'v4': ['v3'], 'v5': ['v4', 'v6'], 'v6': ['v2']}
PageRank Results:
v1: 0.20046031173347453
v2: 0.17648393695535605
v3: 0.16335010147737383
v4: 0.15578531260040415
v5: 0.17343073839836537
v6: 0.17500187633153952

Networkx PageRank Results:
v1: 0.21468970874195084
v3: 0.2536631864577143
v4: 0.16166931067686457
v2: 0.19266842589894956
v5: 0.10688343909265172
v6: 0.07042592913186885


In [ ]:
##################Question 3a######################
from collections import defaultdict
import time

edges = [('v1', 'v2', 7), ('v1', 'v3', 8), ('v2', 'v3', 3), ('v2', 'v4', 2),
         ('v3', 'v4', 1), ('v2', 'v5', 4), ('v4', 'v6', 1), ('v5', 'v6', 1)]

G = nx.Graph()
for edge in edges:
    u, v, weight = edge
    G.add_edge(u, v, weight=weight)
# Implemented the brandes_betweenness_centrality algorithm.
def brandes_betweenness_centrality(G):
    betweenness = defaultdict(float)
    for node in G.nodes():
        stack, predecessors, sigma, distance = [], defaultdict(list), defaultdict(int), defaultdict(int)
        sigma[node], distance[node], queue = 1, 0, [node]
        while queue:
            current_node = queue.pop(0)
            stack.append(current_node)
            for neighbor in G.neighbors(current_node):
                if distance[neighbor] == 0:
                    queue.append(neighbor)
                    distance[neighbor] = distance[current_node] + 1
                if distance[neighbor] == distance[current_node] + 1:
                    sigma[neighbor] += sigma[current_node]
                    predecessors[neighbor].append(current_node)

        delta = defaultdict(float)
        while stack:
            current_node = stack.pop()
            for predecessor in predecessors[current_node]:
                delta[predecessor] += (sigma[predecessor] / sigma[current_node]) * (1 + delta[current_node])
            if current_node != node:
                betweenness[current_node] += delta[current_node]

    return dict(betweenness)
#Normalize data with the same equation displayed in the networkx documentation https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality.html
def normalize(centrality_dict):
    n = len(centrality_dict)
    normalization_factor = 1 / ((n - 1) * (n - 2))
    normalized_centrality = {node: value * normalization_factor for node, value in centrality_dict.items()}
    return normalized_centrality

start_time = time.perf_counter()
betweenness_centrality = brandes_betweenness_centrality(G)
normalized_manual_betweenness = normalize(betweenness_centrality)
end_time = time.perf_counter()
print("Time to implement my betweenness centrality algorithm: {}".format(end_time - start_time))
start_time2 = time.perf_counter()
betweenness_centrality_networkx = nx.betweenness_centrality(G, weight='weight', normalized=False)
normalized_networkx_betweenness = normalize(betweenness_centrality_networkx)
end_time2 = time.perf_counter()
print("Time to implement networkx betweenness centrality algorithm: {}".format(end_time2 - start_time2))

print("Node\tManual Betweenness")
for node in G.nodes():
    print(f"{node}\t{normalized_manual_betweenness[node]:.2f}")
    
print("Node\tNetworkX Betweenness")
for node in G.nodes():
    print(f"{node}\t {normalized_networkx_betweenness[node]:.2f}")

Time to implement my betweenness centrality algorithm: 0.00013379199663177133
Time to implement networkx betweenness centrality algorithm: 0.00018833298236131668
Node	Manual Betweenness
v1	0.02
v2	0.43
v3	0.12
v4	0.26
v5	0.11
v6	0.08
Node	NetworkX Betweenness
v1	 0.00
v2	 0.08
v3	 0.07
v4	 0.28
v5	 0.00
v6	 0.16


In [3]:

file = open("Cit-HepTh.txt", "r")
lines = file.readlines()
file.close()

g4 = nx.DiGraph()

for line in lines[4:]:
    v, u = line.strip().split("\t")
    g4.add_edge(int(v), int(u))

hub, auth = nx.hits(g4)

hubs = sorted(hub, key=hub.get, reverse=True)[:3]
auths = sorted(auth, key=auth.get, reverse=True)[:3]

in_degrees = dict(g4.in_degree())
out_degrees = dict(g4.out_degree())

print("Papers with highest hubs:")
for paper_id in hubs:
    print("Paper ID:", paper_id)
    print("Out-Degree:", out_degrees[paper_id])
    print("----------------------")

print("\nPapers with highest authorities:")
for paper_id in auths:
    print("Paper ID:", paper_id)
    print("In-Degree:", in_degrees[paper_id])
    print("----------------------")


Papers with highest hubs:
Paper ID: 9905111
Out-Degree: 562
----------------------
Paper ID: 110055
Out-Degree: 302
----------------------
Paper ID: 7170
Out-Degree: 263
----------------------

Papers with highest authorities:
Paper ID: 9711200
In-Degree: 2414
----------------------
Paper ID: 9802150
In-Degree: 1775
----------------------
Paper ID: 9802109
In-Degree: 1641
----------------------
